In [1]:
import datetime
from functools import reduce
import regex as re
import warnings
import arrow
import json
import os
import requests
import numpy as np
import math
from google.cloud import bigquery
import pandas as pd
from datetime import datetime as dt

In [2]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    'thermosphr_key.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

 

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

project_id = 'thermosphr-prod'
client_id = 'gera-arcaden-germany'
device_id = 'ayQAWQgc'

 

 

start_time = '2022-12-01'
end_time = '2022-12-02'

 

#client = bigquery.Client(project = project_id)
query = client.query(
            f"""
            SELECT STRING(time_local) AS time_local,
            alias,value, date_update, propertyId
            FROM `{project_id}.bi.datamart_v2` ('{device_id}','{start_time}','{end_time}')
            WHERE alias IS NOT NULL
            AND time_local >= '{start_time}'
            AND time_local < '{end_time}'
            AND client_id = '{client_id}'
            ORDER BY time_local 
                """
        )
dataframe = query.to_dataframe()

 

 

dataframe['value'] = dataframe['value'].astype('float')
dff2 = dataframe[['time_local', 'alias', 'value']].copy()
data_rd = pd.pivot_table(dff2, index='time_local', columns='alias', values='value').reset_index()
data_rd.head()



alias,time_local,ahu_10_air_discharge_flow_sensor,ahu_10_air_discharge_flow_sp,ahu_10_air_discharge_temp_sensor,ahu_10_air_discharge_temp_sp,ahu_10_air_return_flow_sensor,ahu_10_air_return_flow_sp,ahu_10_air_return_temp_sensor,ahu_10_chilled_water_valve_sp,ahu_10_co2_return_sensor,...,precipitation,pressure,snow_depth,specific_humidity,visibility,wetBulb,wind_direction,wind_speed,wind_speed_max,zenith_angle
0,2022-12-01 01:00:00,0.0,0.0,19.246119,15.5,0.0,0.0,18.024424,0.0,428.914886,...,0.0,1024.9100,0.0,0.003401,24.14,-0.282678,283.3600,0.32,0.3000,148.651836
1,2022-12-01 01:15:00,0.0,0.0,19.188492,15.5,0.0,0.0,17.989847,0.0,426.015625,...,0.0,1024.9425,0.0,0.003398,24.14,-0.300238,294.0075,0.35,0.3275,147.457077
2,2022-12-01 01:30:00,0.0,0.0,19.200016,15.5,0.0,0.0,17.966795,0.0,427.147064,...,0.0,1024.9750,0.0,0.003395,24.14,-0.317798,304.6550,0.38,0.3550,146.055529
3,2022-12-01 01:45:00,0.0,0.0,19.153915,15.5,0.0,0.0,17.966795,0.0,428.985596,...,0.0,1025.0075,0.0,0.003392,24.14,-0.335358,315.3025,0.41,0.3825,144.473877
4,2022-12-01 02:00:00,0.0,0.0,19.119339,15.5,0.0,0.0,17.989847,0.0,429.763458,...,0.0,1025.0400,0.0,0.003390,24.14,-0.352918,325.9500,0.44,0.4100,142.737577


In [3]:
import datetime
import pandas as pd
from functools import reduce
import regex as re
import warnings
import arrow
import json
import numpy as np
import requests
import math
# supress warnings to debugging
warnings.filterwarnings("ignore")


class geFramework():
    def __init__(self, rule, batch):
        print(rule.rule_name)
        # get subset of data, min-time and max-time for the given start and end time
        # batch = self.__batch_subset(batch, rule['duration_hours'])
        if 'missing_values' not in rule['rule_name']:
            batch.fillna(method='ffill', inplace=True)
        # indices of the rule dataframe
        alias_cols = rule.index
        subs = 'alias'
        # get columns of rule book that alias in their names
        self.__res = [
            alias_col for alias_col in alias_cols if subs in alias_col]
        self.__rule = rule

        # create empty dataframe for the logs
        self.__df_logs = pd.DataFrame(columns=[
            'rule_name', 'aliases', 'criticality', 'time_local', 'test', 'date_update','qa'])
        aliases = []
        # get a list of aliases for the given targets in the rule book and append that lists
        # in the aliases list

        # in case alias_a is not null
        if rule[self.__res[0]] is not None:

            for i in self.__res:
                if rule[i] is not None and '-' not in rule[i]:
                    # get all aliases for given keywords in config columns
                    aliases.append(self.__alias_substrings(
                        batch, [rule[i]], col_name='alias'))

                # In case minus sign is found to exclude that given alias
                elif rule[i] is not None and '-' in rule[i]:
                    keyword = rule[i].replace('-', '').strip()
                    # get all aliases for given keyword in config columns
                    excluded_aliases = self.__alias_substrings(
                        batch, [keyword], col_name='alias')
                    # get all columns(aliases) of the dataframe
                    unique_aliases = batch.columns.to_list()
                    # drop the given keyword's aliases and time_local from all aliases in dataframe
                    required_aliases = list(
                        set(unique_aliases) - set(excluded_aliases) - set(['time_local']))
                    # append required aliases in the aliases list
                    aliases.append(required_aliases)
#             print(aliases)
            # in case there is only one group of aliases in aliases list
            if len(aliases) == 1:
                # iterate over all group of  alaises
                for alias in aliases[0]:
#                     print(alias)
                    # get a dataframe for each alias
                    df = self.alias_dataframe(self, data_rd=batch, aliases=alias, kpi=rule['rule_name'],
                                              resample_grain=None, daily_resample=None, agg=None, non_negative=None)
#                     display(df)
                    # in case dataframe is not empty
                    if df.shape[0] > 0:
                        # apply the equation that is given in the rule book (equation_1)
                        df = self.apply_equation_1(
                            self, df, equation=rule['equation_1'], temp=True).reset_index(drop=True)
#                         print(df)
                        df = self.apply_equation_2(
                            self, df, rule, equation=rule['equation_2'])
                        # create logs
                        df1 = self.__logs_data(df, rule, [alias])
                        self.__df_logs = pd.concat(
                            [self.__df_logs, df1], axis=0)

                    # in case dataframe is empty
                    else:
                        print(f'Data is not available for {alias}')
            # if in case there are multiple aliases groups in aliases list
            else:

                # create a list of different pairs of aliases
                alias_pair = self.__alias_pairs(self, batch, rule, aliases)

                # iterate for all pairs
                for pair in alias_pair:
                    # in case length of a pair is not equal to 0
                    if len(pair) != 0:
                        # get a dataframe for each pair
                        df = self.alias_pairs_dataframe(self, data_rd=batch, aliases=pair, kpi=rule['rule_name'],
                                                        resample_grain=None, daily_resample=None, agg=None,
                                                        non_negative=None)
                        # in case dataframe is not empty
                        if df.shape[0] > 0:
                            # apply the equation that is given in the rule book (equation_1)
                            df = self.apply_equation_1(
                                self, df, equation=rule['equation_1'], temp=True).reset_index(drop=True)

                            df = self.apply_equation_2(
                                self, df, rule, equation=rule['equation_2'])
                            #                     # create logs
                            df1 = self.__logs_data(df, rule, pair)
                            self.__df_logs = pd.concat(
                                [self.__df_logs, df1], axis=0)
                    # in case dataframe is empty
                    else:
                        print(f'Data is not available for {pair}')
        # if in case alias_a is null
        else:
            # get all columns (aliases) of the dataframe except time_local or time
            aliases.append(self.__alias_substrings(
                batch, None, col_name='alias'))
            # iterate for all aliases
            for alias in aliases[0]:
                # get a dataframe for each pair
                df = self.alias_dataframe(self, data_rd=batch, aliases=alias, kpi=rule['rule_name'],
                                          resample_grain=None, daily_resample=None, agg=None, non_negative=None)
                # in case dataframe is not empty
                if df.shape[0] > 0:
                    # apply the equation that is given in the rule book (equation_1)
                    df = self.apply_equation_1(
                        self, df, equation=rule['equation_1'], temp=True).reset_index(drop=True)

                    df = self.apply_equation_2(
                        self, df, rule, equation=rule['equation_2'])
                    #                     # create logs
                    df1 = self.__logs_data(df, rule, [alias])
                    self.__df_logs = pd.concat([self.__df_logs, df1], axis=0)
                # in case dataframe is empty
                else:
                    print(f'Data is not available for {alias}')

    @staticmethod
    def __logs_data(df=None, rule=None, alias_list=None, result=None):
        """ Gets resultant dataframe and rules for each kpi and creates logs
            in the form of a dataframe.
        Parameters
        ----------
        df: dataframe
        Resultant dataframe for each kpi
        rule:dataframe
        Its the config file
        tgt_alias:string
        Name of the target alias
        ref_alias: string
        Name of the reference alias
        """
        # create an empty dataframe
        df1 = pd.DataFrame()

        # concate all aliases as one string
        if len(alias_list) == 1:
            logs_alias = alias_list[0]
        else:
            logs_alias = ",".join(alias_list)

        # get current utc time (function running time) for date_update
        current_time = datetime.datetime.utcnow().replace(second=0, microsecond=0)

        df1 = df[['test', 'time_local']].copy()
        df1['aliases'] = logs_alias
        df1['rule_name'] = rule['rule_name']
        df1['criticality'] = rule['criticality']
        df1['date_update'] = current_time
        if rule.mute_alert is not None:
            for x in rule.mute_alert[0].split(','):
                df1['mute_alert'] = np.where(df1.aliases.str.contains(x),True,False)
        else:
            df1['mute_alert'] = False
        return df1

    @staticmethod
    def count(df, max_delta):

        try:
            count_true = df['value'].value_counts()[True]
        except:
            count_true = 0

        return count_true

    @staticmethod
    def streak(df):
        """This function calculates delta for only true values, in case
        this calculated delta is greater or equal to the max_delta then
        test will fail otherwise pass.
        Parameters
        ----------
        df : dataframe
        Dataframe after applying equation
        rule: dataframe
        Each row of the config file as a dataframe
        Returns
        -------
        df: dataframe
        delta: float
        """

        # set True value to 1 and False to 0
        df.loc[df["value"] == True, "value"] = 1
        df.loc[df["value"] == False, "value"] = 0
        # Check that original values are not equal to shifted values
        df['expected_streak'] = df.value.ne(df.value.shift())
        # get timelocal for the start of streak
        df['streak_time'] = df.loc[df.expected_streak == True]['time_local']
        # convert streak start time to datetime
        df['streak_time'] = pd.to_datetime(df['streak_time'])
        # converte time_local to datetime
        df['time_local'] = pd.to_datetime(df['time_local'])
        # fill null values from previous values by using forward filling
        df = df.fillna(method='ffill')
        # calculate delta in minutes
        df['delta'] = (df['time_local'] - df['streak_time']
                       ).dt.total_seconds() / 60

        return df

    @staticmethod
    def apply_equation_1(self, df=None, equation=None, temp=None):
        """ This function basically takes a dataframe and an equation
        and then tries to apply this equation and results in the form of boolean.
        Parameters
        ----------
        df: Dataframe
        A Dataframe for an aliases or a pair of aliases
        equation: String
        This string has a set of instruction
        Returns
        -------
        df: Dataframe
        """

        def func(x):
            y = eval(equation)
            return y

        if df.shape[0] > 0:
            # in case equation is given and not missing_values or constant values
            if equation is not None and equation != 'missing_values' and equation != 'constant_values':
                # iterate for given given alias columns in the config file
                for col in self.__res:
                    # add dataframe with alias column
                    if col in equation:
                        col_name = f"x['{col}']"
                        equation = equation.replace(col, col_name)
                # in case time_local is in equation then add dataframe with it
                if 'time_local' in equation:
                    equation = equation.replace(
                        'time_local', 'x["time_local"]')
                    # in case date (2022-08-01) is given with time_local
                    if 'date' in equation:
                        # add double quotes arround the date
                        date = re.findall(r'\d{4}\-\d{2}\-\d{2}', equation)[0]
                        equation = re.sub(date, '"' + date + '"', equation)
                        # convert datetime to string for comparison
                        equation = equation.replace(
                            'x["time_local"].date', 'str(x["time_local"])')
                    df['time_local'] = pd.to_datetime(df['time_local'])
                print(df.shape)
                # apply equation
                df['value'] = df.apply(func, axis=1)
#                 df['value'] = eval(equation)
            # in case equation is None
            else:
                df['value'] = df['alias_a']

        return df

    @staticmethod
    def apply_equation_2(self, df=None, rule=None, equation=None, kpi=None):
        if df.shape[0] > 0:
            if equation is not None:
                if 'streak' in equation:
                    df = self.streak(df)
                    duration = float(re.findall("[0-9.]+", equation)[0])
                    frac,whole = math.modf(duration)
                    duration = (whole*60-15 if whole != 0 else 0) + (frac*60)
                    equation = f'(df.delta >={duration}) & (df.value == True)'
                    df['test'] = eval(equation)

                else:
                    aggregation = re.findall('\w+', equation)[0]
                    operator = re.findall("[^a-zA-Z0-9_.,\s\(\)\[\]]+", equation)[0]
                    window = rule['duration_hours']
                    duration = str(window) + 'h'
                    threshold = float(re.findall("[0-9]+", equation)[0])
                    df['time_local'] = pd.to_datetime(df['time_local'])
                    df.set_index('time_local', inplace=True)
                    df['test'] = eval(f"df['value'].rolling('{duration}',min_periods = {window} * 4 ).agg('{aggregation}') {operator} {threshold}")
                    df.reset_index(inplace=True)
            else:
                df.self.streak(df)
                duration = int(re.findall("[0-9]+", equation)[0]) * 60 - 15
                equation = f'(df.delta >={duration}) & (df.value == True)'
                df['test'] = eval(equation)

            df = df.loc[df.test == True]
            df.test = 'fail'

        return df

    @staticmethod
    def __alias_substrings(df, in_list, col_name='alias'):
        """ This function returns a list of aliases for a given reference in the
            the config file.
        Parameters
        ----------
        df: dataframe
        It's the source dataframe
        in_list: list
        It's the list of given aliases in the config file
        col_name: string
        It's the column name in the source dataframe
        Returns
        -------
        filtered_aliases: list
        This list contains the aliases that are filtered out
        """

        groups = in_list

        if in_list is not None:
            filtered_aliases = []
            substrings = []

            # Creating list of substrings
            for group_id in groups:
                # print(group_id)
                substrings.append(group_id.split('_'))

            unique_aliases = df.columns
            # print(unique_aliases)
            # Removing None from the list of unique aliases
            unique_aliases = list(filter(None, unique_aliases))
            # Filter the aliases that are to be used for the KPI
            for alias in unique_aliases:
                for sub in substrings:
                    if all(a.strip() in alias for a in sub):
                        # print(alias)
                        filtered_aliases.append(alias)
        else:

            remove_cols = ['time_local']
            filtered_aliases = list(set(df.columns.to_list()) - set(remove_cols))

        return filtered_aliases

    @staticmethod
    def __batch_subset(batch, duration):
        """
        Gets a dataframe and a specific duration, where it takes a subset
        of dataframe that falls in the specific hours of duration.
        Parameters
        ----------
        batch: dataframe
        It's the main dataframe whose subset is to be taken based on time column.
        duration:
        A time period for which a subset of the dataframe
        will be taken.
        Returns
        -------
        batch_subset: dataframe
        A subset of the main dataframe for which timing is in between
        the specific duration of hours.
        batch_min_time: time stamp
        It's the start time of the subset dataframe
        batch_max_time: time stamp
        It's the end time of the subset dataframe.
        """
        # set min and max time variable as global
        global batch_min_time
        global batch_max_time

        # get maximum timestamp of the batch
        batch_max_time = str(arrow.get(batch['time_local'].max()).datetime).split('+')[0]
        # convert string formated timestamp to datetime object
        batch_max_time = datetime.datetime.strptime(batch_max_time, '%Y-%m-%d %H:%M:%S')

        # get minimum timestamp by considering duration hours
        batch_min_time = batch_max_time - datetime.timedelta(hours=int(duration),
                                                             minutes=int(round(duration % 1, 4) * 60))
        # get subset of data for the minimum and maximum timestamps
        batch_subset = batch[(batch.time_local >= str(batch_min_time)) & (batch.time_local <= str(batch_max_time))]

        return batch_subset

    @staticmethod
    def alias_pairs_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                              daily_resample=None, agg=None, non_negative=None):
        # create an empty dataframe
        df_master = pd.DataFrame()
        # get indices of aliases's elements
        res = [index for index, val in enumerate(aliases)]
        # create copy of the aliases
        alias_cols = aliases.copy()
        # append time_local into the copied alaises
        alias_cols.append('time_local')
        # get config file aliases columns having same index as res list's elements
        cols = [val for index, val in enumerate(self.__res) if index in res]
        # append time_local into the cols list
        cols.append('time_local')
        # get data for the given aliases
        df = data_rd[alias_cols].copy()
        # assign config file alias column name
        df.columns = cols
        # in case there is data for the pair
        if df.shape[0] > 0:
            # assign df to df_master
            df_master = df
        return df_master

    @staticmethod
    def alias_dataframe(self, data_rd=None, aliases=None, kpi=None, resample_grain=None,
                        daily_resample=None, agg=None, non_negative=None):
        df_master = pd.DataFrame()
        df = data_rd[['time_local', aliases]].copy()
        df.rename(columns={aliases: 'alias_a'}, inplace=True)
        if df.shape[0] > 0:
            df_master = df

        return df_master

    @staticmethod
    def __alias_pairs(self, data_rd, rule, aliases):
        # get indices(numbers) of the internal lists
        self.__index = [index for index, val in enumerate(aliases)]
        # assign config columns names list 
        col_names = self.__res
        # keep those columns names whose index is available in __index list
        col_names = [val for index, val in enumerate(col_names) if index in self.__index]
        # keep index of null list in aliases list
        null_list_index = [index for index, val in enumerate(aliases) if len(val) == 0]
#         self.__index = [index for index, val in enumerate(aliases)]
#         col_names = self.__res
#         col_names = [val for index, val in enumerate(col_names) if index in self.__index]
#         null_list_index = [index for index, val in enumerate(aliases) if len(val) == 0]
        non_null_dfs_list = []
        null_dfs_list = []

#         print('aliases length :: ', len(aliases))
        for i in range(len(aliases)):

            non_null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            null_alias_df = pd.DataFrame(columns=['alias', 'keys'])
            if len(aliases[i]) > 0:
                for alias in aliases[i]:
                    keys = alias.split("_")
                    for ele in rule[col_names[i]].split('_'):
                        keys.remove(ele)

                    if len(keys) == 0:

                        df = pd.DataFrame([[alias, None]], columns=['alias', 'keys'])

                        null_alias_df = pd.concat([null_alias_df, df], axis=0, ignore_index=True)


                    else:

                        key = '_'.join(map(str, keys))
                        df = pd.DataFrame([[alias, key]], columns=['alias', 'keys'])
                        non_null_alias_df = pd.concat([non_null_alias_df, df], axis=0, ignore_index=True)

            if non_null_alias_df.shape[0] > 0:
                non_null_dfs_list.append(non_null_alias_df)
            if null_alias_df.shape[0] > 0:
                null_dfs_list.append(null_alias_df)

        if len(non_null_dfs_list) > 1:
            non_nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, on=["keys"], how="inner"),
                                        non_null_dfs_list)
        elif len(non_null_dfs_list) == 1:
            non_nulls_df_merge = non_null_dfs_list[0]
        else:
            non_nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        if len(null_dfs_list) > 1:
            nulls_df_merge = reduce(lambda left, right: pd.merge(left, right, how="cross"), null_dfs_list)
        elif len(null_dfs_list) == 1:
            nulls_df_merge = null_dfs_list[0]
        else:
            nulls_df_merge = pd.DataFrame(columns=['alias', 'keys'])

        if nulls_df_merge.shape[0] > 0 and non_nulls_df_merge.shape[0] > 0:
            alias_pairs_df = pd.merge(nulls_df_merge, non_nulls_df_merge, how='cross')
        elif nulls_df_merge.shape[0] == 0:
            alias_pairs_df = non_nulls_df_merge
        elif non_nulls_df_merge.shape[0] == 0:
            alias_pairs_df = nulls_df_merge
        
        subs = 'keys'
        keys_cols = [col for col in alias_pairs_df if subs in col]
        alias_pairs_df = alias_pairs_df.drop(keys_cols, axis=1)
        print('columns length => ',len(col_names))


        if len(col_names) == alias_pairs_df.shape[1]:
            alias_pairs = alias_pairs_df.values.tolist()
        else:
            alias_pairs = []

        return alias_pairs

    @staticmethod
    def message_builder(logs, data_start_time, data_end_time, data, PROJECT_ID, logs_link):
        """
        Gets a dataframe, that has information about the tests that are either passed of failed
        Parameters
        ----------
        logs_df: Dataframe
        This is the dataframe, which contains different tests that are performed.
        Returns
        -------
        message: string
        It contains
        """
        current_time = str(datetime.datetime.utcnow().replace(second=0, microsecond=0)) + ' UTC'
        message = f"*{PROJECT_ID}*  {data_start_time} --> {data_end_time}\n"
        if (len(logs.loc[logs.test == 'fail']) > 0):
            df_false = logs.loc[logs['test'] == 'fail']
            rule_names = df_false.loc[df_false.criticality == 'high']['rule_name'].unique()
            for i in range(len(rule_names)):
                message = message + f"\n *{rule_names[i]}*"

            if data.shape[0] > 0:
                message = message + f"\nDuplicate aliases"
                message = message + f"\n {data}"
            else:
                message = message + f"\n No duplicates"

            message = message + f"""\n
    <{logs_link}>
            """
        return message

    def slack_alert(message, slack_webhook):
        """
        Gets a string of information about the tests are failed having only high criticality, and
        send this information to a slack channel.
        Parameters
        ----------
        message: string
        Information about the tests that have high criticality.
        Returns
        -------
        This function returns nothing
        """

        webhook_url = slack_webhook
        slack_data = {'text': message}

        response = requests.post(
            webhook_url, data=json.dumps(slack_data),
            headers={'Content-Type': 'application/json'}
        )
        if response.status_code != 200:
            raise ValueError(
                'Request to slack returned an error %s, the response is:\n%s'
                % (response.status_code, response.text)
            )

    def return_log(self):
        return self.__df_logs

In [9]:

with open('D:\Thermosphr\QA & KPI\QA\QA\qa_config.json') as file:
    config  = json.loads(file.read())
    file.close()
rules = config['rules']
rules = pd.DataFrame(rules)
rules['mute_logs'] = None
rules= rules[rules.rule_name=='ahu_enabled_24h']
rules

,rule_name,alias_a,alias_b,alias_c,alias_d,equation_1,equation_2,duration_hours,criticality,start_date,end_date,mute_alert,mute_logs,qa
1,ahu_enabled_24h,ahu_enable_sensor,None,None,None,alias_a > 0,streak >= 24,24,low,None,None,None,None,hvac_qa


In [10]:
def create_logs(rule, data_rd, logs, templogs):
    obj = geFramework(rule, data_rd)

    if obj is not None:
        dff = obj.return_log()
        logs = pd.concat([logs, dff], axis=0)

        dff = dff.reset_index(drop=True)
        if rule['mute_alert'] is None:

            templogs = pd.concat([templogs, dff], axis=0)

        else:
            aliases_list = rule['mute_alert'].split(',')

            for j in aliases_list:
                dff = dff[~dff.aliases.str.contains(j.strip())]

            templogs = pd.concat([templogs, dff], axis=0)

        if rule['mute_logs'] is None:

            templogs = pd.concat([templogs, dff], axis=0)
            clear_logs = logs
        elif rule['mute_logs'].lower() == 'all':
            dff = dff[~dff.rule_name.str.contains(rule['rule_name'])]

            clear_logs  = logs[~logs.rule_name.str.contains(rule['rule_name'])]
            templogs = pd.concat([templogs, dff], axis=0)
        else:
            aliases_list = rule['mute_logs'].split(',')

            for j in aliases_list:
                dff = dff[~dff.aliases.str.contains(j.strip())]
                clear_logs = logs[~logs.aliases.str.contains(j.strip())]

            templogs = pd.concat([templogs, dff], axis=0)


    return clear_logs, templogs
    
logs = pd.DataFrame()
templogs = pd.DataFrame()
date = arrow.utcnow().date()
date = date.replace(year=1900)


for i in range(rules.shape[0]):
    if not rules.iloc[i]['mute_logs']:
        start_date = rules.iloc[i]['start_date']
        end_date = rules.iloc[i]['end_date']

        if start_date is None and end_date is None:

            logs, templogs = create_logs(rules.iloc[i], data_rd, logs, templogs)

        elif start_date is not None and end_date is not None:

            start_date = dt.strptime(start_date, '%m-%d').date()
            end_date = dt.strptime(end_date, '%m-%d').date()

            if start_date < end_date:
                if start_date <= date <= end_date:
                    logs, templogs = create_logs(rules.iloc[i], data_rd, logs, templogs)
            elif start_date > end_date:
                if start_date >= date and end_date >= date:
                    logs, templogs = create_logs(rules.iloc[i], data_rd, logs, templogs)
            else:
                print('Logs not created for this rule  :  ', rules.iloc[i]['rule_name'])

ahu_enabled_24h


In [11]:
logs

,rule_name,aliases,criticality,time_local,test,date_update,qa


In [248]:
data_rd.head()

alias,time_local,ahu_10_air_discharge_flow_sensor,ahu_10_air_discharge_flow_sp,ahu_10_air_discharge_temp_sensor,ahu_10_air_discharge_temp_sp,ahu_10_air_return_flow_sensor,ahu_10_air_return_flow_sp,ahu_10_air_return_temp_sensor,ahu_10_chilled_water_valve_sp,ahu_10_co2_return_sensor,...,precipitation,pressure,snow_depth,specific_humidity,visibility,wetBulb,wind_direction,wind_speed,wind_speed_max,zenith_angle
0,2022-12-01 01:00:00,0.0,0.0,19.246119,15.5,0.0,0.0,18.024424,0.0,428.914886,...,0.0,1024.9100,0.0,0.003401,24.14,-0.282678,283.3600,0.32,0.3000,148.651836
1,2022-12-01 01:15:00,0.0,0.0,19.188492,15.5,0.0,0.0,17.989847,0.0,426.015625,...,0.0,1024.9425,0.0,0.003398,24.14,-0.300238,294.0075,0.35,0.3275,147.457077
2,2022-12-01 01:30:00,0.0,0.0,19.200016,15.5,0.0,0.0,17.966795,0.0,427.147064,...,0.0,1024.9750,0.0,0.003395,24.14,-0.317798,304.6550,0.38,0.3550,146.055529
3,2022-12-01 01:45:00,0.0,0.0,19.153915,15.5,0.0,0.0,17.966795,0.0,428.985596,...,0.0,1025.0075,0.0,0.003392,24.14,-0.335358,315.3025,0.41,0.3825,144.473877
4,2022-12-01 02:00:00,0.0,0.0,19.119339,15.5,0.0,0.0,17.989847,0.0,429.763458,...,0.0,1025.0400,0.0,0.003390,24.14,-0.352918,325.9500,0.44,0.4100,142.737577


In [478]:
#Task 1
data_rd
data_rd.head()
#config_list=['a_bc','j_kl','ahu_enable_sensor', 'chiller_enable_sensor']
df2 = data_rd.filter(regex = 'ahu.*enable.*cmd|time_local')
unique_aliases = list(df2.columns)
unique_aliases.remove('time_local')

for column in unique_aliases:
    if column == 'ahu_2_fan_air_return_enable_cmd':
        df_new= df2[['time_local',column]]
        #display(df_new.head()) 

        
        
df_new['time_local'] = pd.to_datetime(df['time_local'])        
df_new['time_in_hours'] = df_new['time_local'].dt.hour 

#time_diff=df_new["time_local"].diff().apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
#df_new.insert(1,"Time_diff", time_diff)
'''
df_new["criticality"] = None 
for i, rows in df_new.iterrows():
    col = rows['time_in_hours']
    if col == 1:
        df_new.loc[i,"criticality"] = "Low"
    elif col ==2:
        df_new.loc[i, "criticality"] = "Medium"
    else:
        df_new.loc[i, "criticality"] = "High"
'''
        
df_new["Status"] = "Fail"

df_new


alias,time_local,ahu_2_fan_air_return_enable_cmd,time_in_hours,Status
0,2022-12-01 01:00:00,0.0,1,Fail
1,2022-12-01 01:15:00,0.0,1,Fail
2,2022-12-01 01:30:00,0.0,1,Fail
3,2022-12-01 01:45:00,0.0,1,Fail
4,2022-12-01 02:00:00,0.0,2,Fail
5,2022-12-01 02:15:00,0.0,2,Fail
6,2022-12-01 02:30:00,0.0,2,Fail
7,2022-12-01 02:45:00,0.0,2,Fail
8,2022-12-01 03:00:00,0.0,3,Fail
9,2022-12-01 03:15:00,0.0,3,Fail


In [482]:
#streaks
df_new['start_of_streak']=df_new.Status.ne(df_new['Status'].shift())
#df_new['streak_id'] =df_new['start_of_streak'].cumsum()

#df_new.drop(['criticality_delta'], axis=1, inplace=True)

df_new['streak_time'] = df_new.loc[df_new.start_of_streak == True]['time_local']      
df_new['streak_time'] = pd.to_datetime(df_new['streak_time'])
#forward filling
df_new['streak_time'].ffill(inplace=True)

#df_new['Delta']=df_new['time_local'].dt.minute - df_new['streak_time'].dt.minute
df_new['Delta'] = (df_new['time_local'] - df_new['streak_time']).dt.total_seconds() / 60

df_new["Criticality"] = None 
for i, rows in df_new.iterrows():
    col = rows['Delta']
    if col <= 60:
        df_new.loc[i,"Criticality"] = "Low"
    elif col > 60 and col <=120 :
        df_new.loc[i, "Criticality"] = "Medium"
    else:
        df_new.loc[i, "Criticality"] = "High"

df_new


alias,time_local,ahu_2_fan_air_return_enable_cmd,time_in_hours,Status,start_of_streak,streak_time,Delta,Criticality
0,2022-12-01 01:00:00,0.0,1,Fail,True,2022-12-01 01:00:00,0.0,Low
1,2022-12-01 01:15:00,0.0,1,Fail,False,2022-12-01 01:00:00,15.0,Low
2,2022-12-01 01:30:00,0.0,1,Fail,False,2022-12-01 01:00:00,30.0,Low
3,2022-12-01 01:45:00,0.0,1,Fail,False,2022-12-01 01:00:00,45.0,Low
4,2022-12-01 02:00:00,0.0,2,Fail,False,2022-12-01 01:00:00,60.0,Low
5,2022-12-01 02:15:00,0.0,2,Fail,False,2022-12-01 01:00:00,75.0,Medium
6,2022-12-01 02:30:00,0.0,2,Fail,False,2022-12-01 01:00:00,90.0,Medium
7,2022-12-01 02:45:00,0.0,2,Fail,False,2022-12-01 01:00:00,105.0,Medium
8,2022-12-01 03:00:00,0.0,3,Fail,False,2022-12-01 01:00:00,120.0,Medium
9,2022-12-01 03:15:00,0.0,3,Fail,False,2022-12-01 01:00:00,135.0,High


In [440]:
df_new.head()

alias,time_local,ahu_2_fan_air_return_enable_cmd,time_in_hours,criticality,Status,start_of_streak,streak_time
0,2022-12-01 01:00:00,0.0,1,Low,Fail,True,2022-12-01 01:00:00
1,2022-12-01 01:15:00,0.0,1,Low,Fail,False,2022-12-01 01:00:00
2,2022-12-01 01:30:00,0.0,1,Low,Fail,False,2022-12-01 01:00:00
3,2022-12-01 01:45:00,0.0,1,Low,Fail,False,2022-12-01 01:00:00
4,2022-12-01 02:00:00,0.0,2,Medium,Fail,False,2022-12-01 01:00:00


In [462]:
df_new.isnull().sum()

alias
time_local                         0
ahu_2_fan_air_return_enable_cmd    0
time_in_hours                      0
criticality                        0
Status                             0
start_of_streak                    0
streak_time                        0
dtype: int64

In [474]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   time_local                       92 non-null     datetime64[ns]
 1   ahu_2_fan_air_return_enable_cmd  92 non-null     float64       
 2   time_in_hours                    92 non-null     int64         
 3   criticality                      92 non-null     object        
 4   Status                           92 non-null     object        
 5   start_of_streak                  92 non-null     bool          
 6   streak_time                      92 non-null     datetime64[ns]
 7   Delta                            92 non-null     float64       
dtypes: bool(1), datetime64[ns](2), float64(2), int64(1), object(2)
memory usage: 5.2+ KB
